In [71]:
import pandas as pd
import json
import re
import csv

In [81]:
#features of interest and their marc21 code
columns = { 
'001': 'id',
'041' : 'language_code',
'049': 'holdings',
'100': 'author_person',
'110': 'author_corporate',
'111': 'author_meeting',
'111': 'author_uniform',
'245': 'title',
'264': 'publication',
'300': 'physical_description',
'600': 'subject_personal',
'610': 'subject_corporate',
'611': 'subject_meeting',
'630': 'subject_uniform_title',
'647': 'subject_named_event',
'648': 'subject_chronological',
'650': 'subject_topical',
'651': 'subject_geographic'
          }

In [73]:
columns.values()

dict_values(['id', 'language_code', 'holdings', 'author_person', 'author_corporate', 'author_uniform', 'title', 'publication', 'physical_description', 'subject_personal', 'subject_corporate', 'subject_meeting', 'subject_uniform_title', 'subject_named_event', 'subject_chronological', 'subject_topical', 'subject_geographic'])

# Json to csv


In [78]:
input_path = 'data/01.json'
output_path = 'data/out_01.csv'

with open(path, encoding="utf8") as f: 
    data = json.load(f)
with open(output_path, 'w', newline='', encoding='utf8') as out: 
    writer = csv.writer(out)
    
    #Write header row: 
    writer.writerow(columns.values())
    
    for record in data['records'].values():
        
        row = [
            record.get('001'),
            record.get('041', {}).get('a'), 
            record.get('049'),
            #list of values for 1XX values 
            record.get('100',{}).get('a'),
            record.get('110',{}).get('a'),
            record.get('111',{}).get('a'),
            record.get('245',{}).get('a'),
            record.get('264',{}).get('a'),
            record.get('300',{}).get('a'),
            #need to get a list of values for each of these
            record.get('600',{}).get('a'),
            record.get('610',{}).get('a'),
            record.get('611',{}).get('a'),
            record.get('630',{}).get('a'),
            record.get('647',{}).get('a'),
            record.get('648',{}).get('a'),
            record.get('650',{}).get('a'),
            record.get('651',{}).get('a'),
        ]
        writer.writerow(row)

In [79]:
df = pd.read_csv(output_path)

In [80]:
df

,id,language_code,holdings,author_person,author_corporate,author_uniform,title,publication,physical_description,subject_personal,subject_corporate,subject_meeting,subject_uniform_title,subject_named_event,subject_chronological,subject_topical,subject_geographic
0,BV000023649,ger,{'a': 'DE-Y7'},NaN,Schiller-Nationalmuseum und Deutsches Literatu...,NaN,"Deutsches Literaturarchiv, Schiller-Nationalmu...",Marbach a. Neckar,"181, 16 S.",NaN,Deutsches Literaturarchiv Marbach,NaN,NaN,NaN,Geschichte,NaN,NaN
1,BV000024214,ger,{'a': 'DE-188'},"Schlink, Wilhelm",NaN,NaN,Jacob Burckhardt und die Kunsterwartung im Vor...,Wiesbaden,27 S.,"Burckhardt, Jacob",NaN,NaN,NaN,NaN,Geschichte 1830-1848,Kunstbetrachtung,Deutschland
2,BV000024767,ger,{'a': 'DE-Y3'},"Burckhardt, Jacob",NaN,NaN,Über das Studium der Geschichte,München,582 S.,"Burckhardt, Jacob",NaN,NaN,NaN,NaN,Weltgeschichte,Weltgeschichte,NaN
3,BV000025578,ger,{'a': 'DE-Y2'},"Herzog, Werner",NaN,NaN,Fitzcarraldo,München u.a.,164 S.,"Herzog, Werner",NaN,NaN,Fitzcarraldo,NaN,NaN,German fiction,NaN
4,BV000026429,ita,{'a': 'DE-Y2'},"Rotili, Mario",NaN,NaN,Filippo Raguzzini nel terzo centenario della n...,Napoli,69 S.,"Rauzzini, Filippo",NaN,NaN,NaN,NaN,NaN,"Architecture, Rococo",Italien
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6003,BV001554556,ita,{'a': 'DE-Y2'},"Angeloni, Francesco",NaN,NaN,Historia di Terni,Roma,289 S.,NaN,NaN,NaN,NaN,NaN,NaN,Geschichte,Terni (Italy)
6004,BV001554561,ita,{'a': 'DE-Y2'},"Angelotti, Pompeo",NaN,NaN,Descrittione della città di Rieti,Roma,123 S.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Rieti (Italy)
6005,BV001554563,ita,{'a': 'DE-Y2'},"Angelucci, Giulio A.",NaN,NaN,Stanze dell'Abate Angelucci con documenti e no...,Pisa,254 S.,NaN,NaN,NaN,NaN,NaN,Geschichte 1250-1816,NaN,Arezzo
6006,BV001554686,ita,{'a': 'DE-83'},"Arnaldi, Enea",NaN,NaN,Delle Basiliche antiche e specialmente di quel...,Vicenza,"VIII, CXXXV S., 8 Taf.","Palladio, Andrea","Basilica palladiana (Vicenza, Italy)",NaN,NaN,NaN,Geschichte -1580,Architekturtheorie,Vicenza


### Subject

In [18]:
#Subject Access fields
subject_columns = df.filter(regex=("^6.*")).columns

In [19]:
#function to extract the name of a subject
def extract_name(text):
    r = re.search(r'(?<=\'a\':\s\')(.*?)(?=\')', text)
    return r.group(1) if r else None


df[subject_columns].apply(lambda x: extract_name(str(x)))

610    Deutsches Literaturarchiv Marbach
648                           Geschichte
689                                 None
600                    Burckhardt, Jacob
650                     Kunstbetrachtung
651                          Deutschland
655                          Festschrift
688       Jacob Burckhardt (1818 - 1897)
630                         Fitzcarraldo
653                     Palladio, Andrea
611                                 None
dtype: object

In [20]:
df

,001,003,005,007,008,035,040,041,049,084,...,800,534,515,730,026,850,538,740,533,535
BV000023649,BV000023649,DE-604,20150724,t,870612s1982 |||| 00||| ger d,{'a': '(DE-599)BVBBV000023649'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ger'},{'a': 'DE-Y7'},"{'a': 'GK 9067', '0': '(DE-625)41263:', '2': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000024214,BV000024214,DE-604,20100324,t,870612s1982 gw a||| |||| 00||| ger d,{'a': '(DE-599)BVBBV000024214'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ger'},{'a': 'DE-188'},"{'a': 'NB 6128', '0': '(DE-625)124171:', '2': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000024767,BV000024767,DE-604,20170322,t,870612s1982 gw |||| 00||| ger d,{'a': '(DE-599)BVBBV000024767'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ger'},{'a': 'DE-Y3'},"{'a': '5,1', '2': 'ssgn'}",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000025578,BV000025578,DE-604,20100903,t,870612s1982 b||| |||| 00||| ger d,{'a': '(DE-599)BVBBV000025578'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ger'},{'a': 'DE-Y2'},"{'a': 'GN 9999', '0': '(DE-625)43130:', '2': '...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV000026429,BV000026429,DE-604,20210505,t,870612s1982 a||| |||| 01||| ita d,{'a': '(DE-599)BVBBV000026429'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},"{'a': 'LI 99999', '0': '(DE-625)97471:', '2': ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
BV001554556,BV001554556,DE-604,20010720,t,890628s1646 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554556'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakwb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554561,BV001554561,DE-604,00000000000000.0,t,890628s1635 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554561'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554563,BV001554563,DE-604,20160826,t,890628s1816 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554563'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-Y2'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
BV001554686,BV001554686,DE-604,20160115,t,890628s1769 |||| 00||| ita d,{'a': '(DE-599)BVBBV001554686'},"{'a': 'DE-604', 'b': 'ger', 'e': 'rakddb'}",{'a': 'ita'},{'a': 'DE-83'},NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### List of columns that we might be interested in
041: 'language_code'  
049: 'holdings'  
100: 'author_person'    
110: 'author_corporate'  
245: 'title'  
264: 'publication'  
300: 'physical_description'  
600: 'subject_personal'  
611: 'subject_corporate'  
648: 'subject_chronological'  
650: 'subject_topical'  
keep all the 6XX for topics  

